# HW4P2: Attention-based Speech Recognition

Welcome to the final assignment in 11785. In this HW, you will work on building a speech recognition system with attention. <br> <br>

HW Writeup: https://piazza.com/class_profile/get_resource/l37uyxe87cq5xn/lam1lcjjj0314e <br>
Kaggle competition link: https://www.kaggle.com/competitions/11-785-f22-hw4p2/ <br>
LAS Paper: https://arxiv.org/pdf/1508.01211.pdf <br>
Attention is all you need:https://arxiv.org/pdf/1706.03762.pdf

# Initial Set-up

In [ ]:
!nvidia-smi

Sun Dec 11 22:06:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install some required libraries
# Feel free to add more if you want
!pip install -q python-levenshtein torchsummaryX wandb

     |████████████████████████████████| 1.9 MB 7.5 MB/s 
     |████████████████████████████████| 174 kB 7.6 MB/s 
     |████████████████████████████████| 2.2 MB 59.0 MB/s 
     |████████████████████████████████| 173 kB 57.0 MB/s 
     |████████████████████████████████| 182 kB 59.0 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 47.8 MB/s 
     |████████████████████████████████| 168 kB 61.7 MB/s 
     |████████████████████████████████| 166 kB 52.5 MB/s 
     |████████████████████████████████| 166 kB 68.0 MB/s 
     |████████████████████████████████| 162 kB 75.2 MB/s 
     |████████████████████████████████| 162 kB 38.7 MB/s 
     |████████████████████████████████| 158 kB 62.9 MB/s 
     |████████████████████████████████| 157 kB 62.9 MB/s 
     |████████████████████████████████| 157 kB 58.0 MB/s 
     |████████████████████████████████| 157 kB 61.9 MB/s 
     |████████████████████████████████| 157 kB 58.4 MB/s 
     |████████████

In [ ]:
!pip install pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 5.0 MB/s 


# Import

In [ ]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", DEVICE)

Device:  cuda


In [ ]:
import os
import pandas as pd
import numpy as np
import Levenshtein

import torch
import torchaudio
from torchnlp.nn import LockedDropout

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import gc
from torchsummaryX import summary
import wandb
from glob import glob
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", DEVICE)

Device:  cuda


In [ ]:
# # TODO: Import drive if you are a colab user
# from google.colab import drive
# drive.mount('/content/drive')

# Config

In [ ]:
# Global config dict. Feel free to add or change if you want.
config = {
    'batch_size': 96,
    'epochs': 60,
    'lr': 1e-3
}

# Toy Data Setup

The toy dataset is very essential for you in this HW. The model which you will be building is complicated and you first need to make sure that it runs on the toy dataset. <br>
In other words, you need convergence - the attention diagonal. Take a look at the write-up for this. <br>
We have given you the following code to download the toy data and load it. You can use it the way it is. But be careful, the transcripts are different from the original data from kaggle. The toy dataset has phonemes but the actual data has characters.

In [ ]:
!wget -q https://cmu.box.com/shared/static/wok08c2z2dp4clufhy79c5ee6jx3pyj9 --content-disposition --show-progress
!wget -q https://cmu.box.com/shared/static/zctr6mvh7npfn01forli8n45duhp2g85 --content-disposition --show-progress
!wget -q https://cmu.box.com/shared/static/m2oaek69145ljeu6srtbbb7k0ip6yfup --content-disposition --show-progress
!wget -q https://cmu.box.com/shared/static/owrjy0tqra3v7zq2ru7mocy2djskydy9 --content-disposition --show-progress

f0176_mfccs_train.n 100%[===================>] 279.30M  12.5MB/s    in 22s     
f0176_mfccs_dev.npy 100%[===================>]  27.93M  13.3MB/s    in 2.1s    
f0176_hw3p2_train.n 100%[===================>]   7.02M  5.21MB/s    in 1.3s    
f0176_hw3p2_dev.npy 100%[===================>] 718.88K   905KB/s    in 0.8s    


In [ ]:
# Load the toy dataset
X_train = np.load("f0176_mfccs_train.npy")
X_valid = np.load("f0176_mfccs_dev.npy")
Y_train = np.load("f0176_hw3p2_train.npy")
Y_valid = np.load("f0176_hw3p2_dev.npy")

# This is how you actually need to find out the different trancripts in a dataset. 
# Can you think whats going on here? Why are we using a np.unique?
VOCAB_MAP           = dict(zip(np.unique(Y_valid), range(len(np.unique(Y_valid))))) 
VOCAB_MAP["[PAD]"]  = len(VOCAB_MAP)
VOCAB               = list(VOCAB_MAP.keys())

SOS_TOKEN = VOCAB_MAP["[SOS]"]
EOS_TOKEN = VOCAB_MAP["[EOS]"]
PAD_TOKEN = VOCAB_MAP["[PAD]"]

Y_train = [np.array([VOCAB_MAP[p] for p in seq]) for seq in Y_train]
Y_valid = [np.array([VOCAB_MAP[p] for p in seq]) for seq in Y_valid]

In [ ]:
# Dataset class for the Toy dataset
class ToyDataset(torch.utils.data.Dataset):

    def __init__(self, partition):

        if partition == "train":
            self.mfccs = X_train[:, :, :15]
            self.transcripts = Y_train

        elif partition == "valid":
            self.mfccs = X_valid[:, :, :15]
            self.transcripts = Y_valid

        assert len(self.mfccs) == len(self.transcripts)

        self.length = len(self.mfccs)

    def __len__(self):

        return self.length

    def __getitem__(self, i):

        x = torch.tensor(self.mfccs[i])
        y = torch.tensor(self.transcripts[i])

        return x, y

    def collate_fn(batch):

        x_batch, y_batch = list(zip(*batch))

        x_lens      = [x.shape[0] for x in x_batch] 
        y_lens      = [y.shape[0] for y in y_batch] 

        x_batch_pad = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value= EOS_TOKEN)
        y_batch_pad = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value= EOS_TOKEN) 
        
        return x_batch_pad, y_batch_pad, torch.tensor(x_lens), torch.tensor(y_lens)

In [ ]:
# Create objects for the dataset class
train_data = ToyDataset(partition="train") #TODO
val_data = ToyDataset(partition="valid") # TODO : You can either use the same class with some modifications or make a new one :)

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory=True,
                                           shuffle=True, collate_fn=ToyDataset.collate_fn)
val_loader = DataLoader(val_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory=True,
                                           shuffle=True, collate_fn=ToyDataset.collate_fn)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

Batch size:  96
Train dataset samples = 16000, batches = 167
Val dataset samples = 1600, batches = 17


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([96, 176, 15]) torch.Size([96, 23]) torch.Size([96]) torch.Size([96])


# Kaggle Data

In [ ]:
# TODO: Use the same Kaggle code from HW1P2, HW2P2, HW3P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle/

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ruidichang","key":"ec93dcec34e885b48bb96b02e9b79b80"}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 4.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=d82691f96fc8674c4cc7462da821884e8b5888f7d68ad5ff674d81dec8fdfcb3
  Stored in directory: /root/.cache/pip/wheels/f3/67/7b/a6d668747974998471d29b230e7221dd01330ac34faebe4af4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# Download the data
!kaggle competitions download -c 11-785-f22-hw4p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw4p2.zip' -d '/content/data'

 99% 2.06G/2.09G [00:14<00:00, 250MB/s]
100% 2.09G/2.09G [00:14<00:00, 152MB/s]


# Dataset Class

In [ ]:
# These are the various characters in the transcripts of the datasetW
VOCAB = ['<sos>',   
         'A',   'B',    'C',    'D',    
         'E',   'F',    'G',    'H',    
         'I',   'J',    'K',    'L',       
         'M',   'N',    'O',    'P',    
         'Q',   'R',    'S',    'T', 
         'U',   'V',    'W',    'X', 
         'Y',   'Z',    "'",    ' ', 
         '<eos>']

VOCAB_MAP = {VOCAB[i]:i for i in range(0, len(VOCAB))}

SOS_TOKEN = VOCAB_MAP["<sos>"]
EOS_TOKEN = VOCAB_MAP["<eos>"]
print(VOCAB_MAP)
print(len(VOCAB))

{'<sos>': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, "'": 27, ' ': 28, '<eos>': 29}
30


In [ ]:
# TODO: Create a dataset class which is exactly the same as HW3P2. You are free to reuse it. 
# The only change is that the transcript mapping is different for this HW.
# Note: We also want to retain SOS and EOS tokens in the transcript this time.
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, VOCAB, path): 

        # Load the directory and all files in them
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.transcript_dir = os.path.join(path,'transcript/raw')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.vocab = VOCAB
        self.mfccs, self.transcripts = [], []

        assert len(self.mfcc_files) == len(self.transcript_files) # Making sure that we have the same no. of mfcc and transcripts

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?

        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir,self.transcript_files[i]))
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
          
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS  
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append([self.vocab.index(t) for t in transcript])


    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.mfccs)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS
        '''
        mfcc = torch.tensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind]) 
        return mfcc, transcript


    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''

        batch_mfcc, batch_transcript = list(zip(*batch))

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True, padding_value= EOS_TOKEN)
        lengths_mfcc = [sample.shape[0] for sample in batch_mfcc]

        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(
            batch_transcript,  batch_first=True, padding_value=EOS_TOKEN)
        lengths_transcript = [sample.shape[0] for sample in batch_transcript]
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

In [ ]:
# TODO: Similarly, create a test dataset class
class AudioDatasetTest(torch.utils.data.Dataset):

    def __init__(self, VOCAB, path): 

        # Load the directory and all files in them
        self.mfcc_dir = os.path.join(path,'mfcc')
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.vocab = VOCAB
        self.mfccs = []

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?

        for i in range(0, self.length):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,self.mfcc_files[i]))
        #   Optionally do Cepstral Normalization of mfcc
            self.mfccs.append(mfcc)


    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return len(self.mfccs)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS
        '''
        mfcc = torch.tensor(self.mfccs[ind])
        return mfcc


    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''

        batch_mfcc = [mfcc for mfcc in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True, padding_value= EOS_TOKEN)
        lengths_mfcc = [sample.shape[0] for sample in batch_mfcc]
        
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

# Dataset and Dataloaders

In [ ]:
transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

path = '/content/data/hw4p2' 

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

26

In [ ]:
# TODO: Create the datasets and dataloaders
# All these things are similar to HW3P2
# You can reuse the same code

# The sanity check for shapes also are similar
# Please remember that the only change in the dataset for this HW is the transcripts
# So you are expected to get similar shapes like HW3P2 (Pad, pack and Oh my!)
train_data = AudioDataset(VOCAB, os.path.join(path,'train-clean-100')) 
val_data = AudioDataset(VOCAB, os.path.join(path,'dev-clean')) # TODO : You can either use the same class with some modifications or make a new one :)

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True,collate_fn=AudioDataset.collate_fn)
val_loader = DataLoader(val_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= False, collate_fn=AudioDataset.collate_fn)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

In [ ]:
test_data = AudioDatasetTest(VOCAB,os.path.join(path,'test-clean'))

test_loader = DataLoader(test_data, num_workers= 2,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= False, collate_fn=AudioDatasetTest.collate_fn)

print("Batch size: ", config['batch_size'])
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([96, 1642, 15]) torch.Size([96, 291]) torch.Size([96]) torch.Size([96])


In [ ]:
# sanity check
for data in test_loader:
    t, lt = data
    print(t.shape, lt.shape)
    break 

torch.Size([96, 1047, 15]) torch.Size([96])


# Model

In this section you will be building the LAS model from scratch. Before starting to code, please read the writeup, paper and understand the following parts completely.<br>
- Pyramidal Bi-LSTM 
- Listener
- Attention
- Speller

After getting a good grasp of the workings of these modules, start coding. Follow the TODOs carefully. We will also be adding some extra features to the attention mechanism like keys and values which are not originally present in LAS. So we will be creating a hybrid network based on LAS and Attention is All You Need.


## Encoder

### Pyramidal Bi-LSTM

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)# TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed, x_lens): # x_packed is a PackedSequence
        # TODO: Pad Packed Sequence
        # x, x_lens = pad_packed_sequence(x_packed, batch_first=True)
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x, x_lens = self.trunc_reshape(x_packed, x_lens)
        # TODO: Pack Padded Sequence. What output(s) would you get?
        x = pack_padded_sequence(x, x_lens/2, batch_first=True, enforce_sorted=False)
        # TODO: Pass the sequence through bLSTM
        output = self.blstm(x)[0]
        output, output_len = pad_packed_sequence(output, batch_first=True)
        # What do you return?
        return output, output_len

    def trunc_reshape(self, x, x_lens): 
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        if(x.shape[1]%2!=0):
            x = x[:,:-1,:]
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x = x.reshape(x.shape[0], int(x.shape[1]/2), x.shape[2]*2)
        # TODO: Reduce lengths by the same downsampling factor
        return x, x_lens

### Listener

In [ ]:
class Listener(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Listener, self).__init__()
        # The first LSTM at the very bottom
        self.blstm = torch.nn.LSTM(input_size=input_size, hidden_size=encoder_hidden_size, num_layers=1, bidirectional=True, batch_first=True)#TODO: Fill this up

        # self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
        #     # TODO: Fill this up with pBLSTMs - What should the input_size be? 
        #     # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
        #     # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
        #     pBLSTM(encoder_hidden_size * 4, encoder_hidden_size),
        #     pBLSTM(encoder_hidden_size * 4, encoder_hidden_size),
        #     pBLSTM(encoder_hidden_size * 4, encoder_hidden_size)
        # )
        self.pBLSTM1 = pBLSTM(encoder_hidden_size * 4, encoder_hidden_size)
        self.pBLSTM2 = pBLSTM(encoder_hidden_size * 4, encoder_hidden_size)
        self.pBLSTM3 = pBLSTM(encoder_hidden_size * 4, encoder_hidden_size)
        self.lockeddropout = LockedDropout(p=0.2)
         
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        # TODO: Pack Padded Sequence
        packed_sequence = pack_padded_sequence(x, x_lens, enforce_sorted=False, batch_first=True)
        # TODO: Pass it through the first LSTM layer (no truncation)
        out, _ = self.blstm(packed_sequence)
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        out, out_lens = pad_packed_sequence(out, batch_first=True)
        encoder_outputs, encoder_lens = self.pBLSTM1(out, out_lens)
        encoder_outputs = self.lockeddropout(encoder_outputs)
        encoder_outputs, encoder_lens = self.pBLSTM2(encoder_outputs, encoder_lens)
        encoder_outputs = self.lockeddropout(encoder_outputs)
        encoder_outputs, encoder_lens = self.pBLSTM3(encoder_outputs, encoder_lens)
        # TODO: Pad Packed Sequence
        # encoder_outputs, encoder_lens = pad_packed_sequence(encoder_outputs, batch_first=True) 
        # Remember the number of output(s) each function returns
        return encoder_outputs, encoder_lens

In [ ]:
encoder = Listener(15,256).to(DEVICE)# TODO: Initialize Listener
print(encoder)
summary(encoder, x.to(DEVICE), lx)
del encoder

Listener(
  (blstm): LSTM(15, 256, batch_first=True, bidirectional=True)
  (pBLSTM1): pBLSTM(
    (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
  )
  (pBLSTM2): pBLSTM(
    (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
  )
  (pBLSTM3): pBLSTM(
    (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
  )
  (lockeddropout): LockedDropout(p=0.2)
)
                     Kernel Shape    Output Shape     Params Mult-Adds
Layer                                                                 
0_blstm                         -   [124214, 512]   559.104k  555.008k
1_pBLSTM1.LSTM_blstm            -    [62081, 512]  2.625536M  2.62144M
2_lockeddropout                 -  [96, 821, 512]          -         -
3_pBLSTM2.LSTM_blstm            -    [31015, 512]  2.625536M  2.62144M
4_lockeddropout                 -  [96, 410, 512]          -         -
5_pBLSTM3.LSTM_blstm            -    [15478, 512]  2.625536M  2.62144M
-----------------------------------------

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


## Attention (Attend)

Different ways to compute Attention

1. Dot-product attention
    * raw_weights = bmm(key, query) 
    * Optional: Scaled dot-product by normalizing with sqrt key dimension 
    * Check "Attention is All You Need" Section 3.2.1
    * 1st way is what most TAs are comfortable with, but if you want to explore, check out other methods below


2. Cosine attention
    * raw_weights = cosine(query, key) # almost the same as dot-product xD 

3. Bi-linear attention
    * W = Linear transformation (learnable parameter): d_k -> d_q
    * raw_weights = bmm(key @ W, query)

4. Multi-layer perceptron
    * Check "Neural Machine Translation and Sequence-to-sequence Models: A Tutorial" Section 8.4

5. Multi-Head Attention
    * Check "Attention is All You Need" Section 3.2.2
    * h = Number of heads
    * W_Q, W_K, W_V: Weight matrix for Q, K, V (h of them in total)
    * W_O: d_v -> d_v
    * Reshape K: (B, T, d_k) to (B, T, h, d_k // h) and transpose to (B, h, T, d_k // h)
    * Reshape V: (B, T, d_v) to (B, T, h, d_v // h) and transpose to (B, h, T, d_v // h)
    * Reshape Q: (B, d_q) to (B, h, d_q // h) `
    * raw_weights = Q @ K^T
    * masked_raw_weights = mask(raw_weights)
    * attention = softmax(masked_raw_weights)
    * multi_head = attention @ V
    * multi_head = multi_head reshaped to (B, d_v)
    * context = multi_head @ W_O

In [ ]:
def plot_attention(attention): 
    # Function for plotting attention
    # You need to get a diagonal plot
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

class Attention(torch.nn.Module):
    '''
    Attention is calculated using the key, value (from encoder hidden states) and query from decoder.
    Here are different ways to compute attention and context:

    After obtaining the raw weights, compute and return attention weights and context as follows.:

    masked_raw_weights  = mask(raw_weights) # mask out padded elements with big negative number (e.g. -1e9 or -inf in FP16)
    attention           = softmax(masked_raw_weights)
    context             = bmm(attention, value)
    
    At the end, you can pass context through a linear layer too.

    '''
    
    def __init__(self, encoder_hidden_size, decoder_output_size, projection_size):
        super(Attention, self).__init__()

        self.key_projection     = torch.nn.Linear(encoder_hidden_size * 2, projection_size)# TODO: Define an nn.Linear layer which projects the encoder_hidden_state to keys
        self.value_projection   = torch.nn.Linear(encoder_hidden_size * 2, projection_size)# TODO: Define an nn.Linear layer which projects the encoder_hidden_state to value
        self.query_projection   = torch.nn.Linear(decoder_output_size , projection_size)# TODO: Define an nn.Linear layer which projects the decoder_output_state to query
        # Optional : Define an nn.Linear layer which projects the context vector

        self.softmax            = torch.nn.Softmax(dim=1)# TODO: Define a softmax layer. Think about the dimension which you need to apply 
        # Tip: What is the shape of energy? And what are those?

    # As you know, in the attention mechanism, the key, value and mask are calculated only once.
    # This function is used to calculate them and set them to self
    def set_key_value_mask(self, encoder_outputs, encoder_lens):
    
        encoder_max_seq_len = encoder_outputs.shape[1]

        self.key      = self.key_projection(encoder_outputs) # TODO: Project encoder_outputs using key_projection to get keys
        self.value    = self.value_projection(encoder_outputs)# TODO: Project encoder_outputs using value_projection to get values

        # encoder_max_seq_len is of shape (batch_size, ) which consists of the lengths encoder output sequences in that batch
        # The raw_weights are of shape (batch_size, timesteps)

        # TODO: To remove the influence of padding in the raw_weights, we want to create a boolean mask of shape (batch_size, timesteps) 
        # The mask is False for all indicies before padding begins, True for all indices after.

        self.padding_mask     =  torch.arange(encoder_max_seq_len).unsqueeze(0) >= encoder_lens.unsqueeze(1)# TODO: You want to use a comparison between encoder_max_seq_len and encoder_lens to create this mask. 
        self.padding_mask     =  self.padding_mask.to(DEVICE)
        # (Hint: Broadcasting gives you a one liner)
        
    def forward(self, decoder_output_embedding):
        # key   : (batch_size, timesteps, projection_size)
        # value : (batch_size, timesteps, projection_size)
        # query : (batch_size, projection_size)

        self.query         = self.query_projection(decoder_output_embedding)# TODO: Project the query using query_projection
        # Hint: Take a look at torch.bmm for the products below 

        raw_weights        = (torch.bmm(self.key, self.query.unsqueeze(2))).squeeze(2)# TODO: Calculate raw_weights which is the product of query and key, and is of shape (batch_size, timesteps)
        _MASKING_VALUE     = -1e+30 if raw_weights.dtype == torch.float32 else -1e+4
        masked_raw_weights = raw_weights.masked_fill_(self.padding_mask, _MASKING_VALUE) # TODO: Mask the raw_weights with self.padding_mask. 
        # Take a look at pytorch's masked_fill_ function (You want the fill value to be a big negative number for the softmax to make it close to 0)

        attention_weights  = self.softmax(masked_raw_weights)# TODO: Calculate the attention weights, which is the softmax of raw_weights
        context            = torch.bmm(attention_weights.unsqueeze(1), self.value).squeeze(1)# TODO: Calculate the context - it is a product between attention_weights and value

        # Hint: You might need to use squeeze/unsqueeze to make sure that your operations work with bmm
        # print("context size:",context.shape)
        return context, attention_weights # Return the context, attention_weights

## Decoder

### Speller

In [ ]:
class Speller(torch.nn.Module):

    def __init__(self, embed_size, decoder_hidden_size, decoder_output_size, vocab_size, attention_module= None):
        super(Speller, self).__init__()

        self.vocab_size         = vocab_size

        self.decoder_output_size = decoder_output_size

        self.embedding          = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)# TODO: Initialize the Embedding Layer (Use the nn.Embedding Layer from torch), make sure you set the correct padding_idx  

        self.lstm_cell1         = torch.nn.LSTMCell(input_size=embed_size+decoder_output_size, hidden_size=decoder_hidden_size)
                                      
        self.lstm_cell2         = torch.nn.LSTMCell(input_size=decoder_hidden_size, hidden_size=decoder_output_size)
        
        self.lockeddropout      = LockedDropout(p=0.2)

        self.char_prob          = torch.nn.Linear(decoder_output_size+decoder_output_size, vocab_size)# TODO: Initialize the classification layer to generate your probability distribution over all characters

        self.char_prob.weight   = self.embedding.weight # Weight tying

        self.attention          = attention_module

    
    def forward(self, encoder_outputs, encoder_lens, y = None, tf_rate = 1): 

        '''
        Args: 
            embedding: Attention embeddings 
            hidden_list: List of Hidden States for the LSTM Cells
        ''' 
      
        batch_size, encoder_max_seq_len, _ = encoder_outputs.shape
        # print(encoder_outputs.shape)
        # print(encoder_lens.shape)
        if self.training:
            timesteps     = y.shape[1] # The number of timesteps is the sequence of length of your transcript during training
            label_embed   = self.embedding(y) # Embeddings of the transcript, when we want to use teacher forcing
        else:
            timesteps     = 600 # 600 is a design choice that we recommend, however you are free to experiment.

        # INITS
        predictions     = []
        prediction      = torch.zeros(batch_size, 1).to(DEVICE)

        # Initialize the first character input to your decoder, SOS
        char            = torch.full((batch_size,), fill_value=SOS_TOKEN, dtype= torch.long).to(DEVICE) 

        # Initialize a list to keep track of LSTM Cell Hidden and Cell Memory States, to None
        hidden_states   = [None,None]

        attention_plot          = []
        context                 = torch.zeros(batch_size, self.decoder_output_size).to(DEVICE)# TODO: Initialize context (You have a few choices, refer to the writeup )
        attention_weights       = torch.zeros(batch_size, encoder_max_seq_len) # Attention Weights are zero if not using Attend Module
        # Set Attention Key, Value, Padding Mask just once
        if self.attention != None:
            self.attention.set_key_value_mask(encoder_outputs, encoder_lens)

        for t in range(timesteps):
            
            if self.training and t > 0:
                # TODO: We want to decide which embedding to use as input for the decoder during training
                # We can use the embedding of the transcript character or the embedding of decoded/predicted character, from the previous timestep 
                # Using the embedding of the transcript character is teacher forcing, it is very important for faster convergence
                # Use a comparison between a random probability and your teacher forcing rate, to decide which embedding to use
                if np.random.random() > tf_rate:
                    prediction = torch.nn.functional.gumbel_softmax(prediction) # Gumbel noise
                    char_embed = self.embedding(prediction.argmax(dim=-1)).to(DEVICE)
                # Teacher Forcing
                else:
                    char_embed = label_embed[:,t-1].to(DEVICE)
            else:
                char_embed = self.embedding(char).to(DEVICE)#TODO: Generate the embedding for the character at timestep t
            # print("char_embed shape", char_embed.shape)
            decoder_input_embedding = torch.cat([char_embed, context], dim=1)# TODO: What do we want to concatenate as input to the decoder? (Use torch.cat)
            
            # print("decoder input shape:", decoder_input_embedding.shape )
            # Loop over your lstm cells
            # Each lstm cell takes in an embedding 
            # print(decoder_input_embedding.shape)

            hidden_states[0] = self.lstm_cell1(decoder_input_embedding, hidden_states[0]) 
            decoder_input_embedding = hidden_states[0][0]
            decoder_input_embedding = self.lockeddropout(decoder_input_embedding.unsqueeze(0)).squeeze(0)
            hidden_states[1] = self.lstm_cell2(decoder_input_embedding, hidden_states[1]) 
            decoder_input_embedding = hidden_states[1][0]
            # The output embedding from the decoder is the hidden state of the last LSTM Cell
            decoder_output_embedding = hidden_states[-1][0]
            
            # We compute attention from the output of the last LSTM Cell
            if self.attention != None:
                context, attention_weights = self.attention(decoder_output_embedding) # The returned query is the projected query

            attention_plot.append(attention_weights[0].detach().cpu())
            output_embedding     = torch.cat([hidden_states[-1][0], context], dim=1)# TODO: Concatenate the projected query with context for the output embedding
            # Hint: How can you get the projected query from attention
            # If you are not using attention, what will you use instead of query?
            # print("shape after cat:",output_embedding.shape)
            # print(output_embedding.shape)
            # print(self.char_prob.weight.shape)
            prediction            = self.char_prob(output_embedding)
            
            # Append the character probability distribution to the list of predictions 
            predictions.append(prediction)
            # print("prediction shape", len(predictions))
            char = torch.argmax(prediction, dim=-1)# TODO: Get the predicted character for the next timestep from the probability distribution 
            # (Hint: Use Greedy Decoding for starters)

        attention_plot  = torch.stack(attention_plot, dim=1)# TODO: Stack list of attetion_plots 
        predictions     = torch.stack(predictions, dim=1)# TODO: Stack list of predictions 
        # print("final shape",predictions.shape)
        return predictions, attention_plot

## Sequence-to-Sequence Model

### LAS

In [ ]:
class LAS(torch.nn.Module):
    def __init__(self, input_size, encoder_hidden_size,
                 vocab_size, embed_size,
                 decoder_hidden_size, decoder_output_size,
                 projection_size= 128):
        
        super(LAS, self).__init__()

        self.encoder        = Listener(input_size, encoder_hidden_size)# TODO: Initialize Encoder
        attention_module    = Attention(encoder_hidden_size, decoder_output_size, projection_size)# TODO: Initialize Attention
        self.decoder        = Speller(embed_size, decoder_hidden_size, decoder_output_size, vocab_size, attention_module = attention_module)# TODO: Initialize Decoder, make sure you pass the attention module 

    def forward(self, x, x_lens, y = None, tf_rate = 1):

        encoder_outputs, encoder_lens = self.encoder(x, x_lens) # from Listener
        predictions, attention_plot = self.decoder(encoder_outputs, encoder_lens, y, tf_rate)
        return predictions, attention_plot

# Training Setup

## Model Setup










In [ ]:
# Baseline LAS has the following configuration:
# Encoder bLSTM/pbLSTM Hidden Dimension of 512 (256 per direction)
# Decoder Embedding Layer Dimension of 256
# Decoder Hidden Dimension of 512 
# Decoder Output Dimension of 128
# Attention Projection Size of 128
# Feel Free to Experiment with this 
model = LAS(
    input_size=15, encoder_hidden_size=512,
                 vocab_size=len(VOCAB), embed_size=256,
                 decoder_hidden_size=512, decoder_output_size=128,
                 projection_size= 128
    # Initialize your model 
    # Read the paper and think about what dimensions should be used
    # You can experiment on these as well, but they are not requried for the early submission
    # Remember that if you are using weight tying, some sizes need to be the same
).to(DEVICE)

summary(model, x= x.to(DEVICE), x_lens= lx, y= y.to(DEVICE))

## Optimizer, Scheduler, Loss

In [ ]:
optimizer   = torch.optim.Adam(model.parameters(), lr= 0.001, amsgrad= True, weight_decay= 5e-6)
criterion   = torch.nn.CrossEntropyLoss(reduction='none') # Why are we using reduction = 'none' ? 
scaler      = torch.cuda.amp.GradScaler()

# Optional: Create a custom class for a Teacher Force Schedule 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Levenshtein Distance

In [ ]:
# We have given you this utility function which takes a sequence of indices and converts them to a list of characters
def indices_to_chars(indices, vocab):
    tokens = []
    for i in indices: # This loops through all the indices
        if int(i) == SOS_TOKEN: # If SOS is encountered, dont add it to the final list
            continue
        elif int(i) == EOS_TOKEN: # If EOS is encountered, stop the decoding process
            break
        else:
            tokens.append(vocab[i])
    return tokens

# To make your life more easier, we have given the Levenshtein distantce / Edit distance calculation code
def calc_edit_distance(predictions, y, ly, vocab= VOCAB, print_example= False):

    dist                = 0
    batch_size, seq_len = predictions.shape

    for batch_idx in range(batch_size): 

        y_sliced    = indices_to_chars(y[batch_idx,0:ly[batch_idx]], vocab)
        pred_sliced = indices_to_chars(predictions[batch_idx], vocab)

        # Strings - When you are using characters from the AudioDataset
        y_string    = ''.join(y_sliced)
        pred_string = ''.join(pred_sliced)
        
        dist        += Levenshtein.distance(pred_string, y_string)
        # Comment the above and uncomment below for toy dataset 
        # dist      += Levenshtein.distance(y_sliced, pred_sliced)

    if print_example: 
        # Print y_sliced and pred_sliced if you are using the toy dataset
        print("Ground Truth : ", y_string)
        print("Prediction   : ", pred_string)
        
    dist/=batch_size
    return dist

# Train and Evaluate

In [ ]:
def train(model, dataloader, criterion, optimizer, teacher_forcing_rate):

    model.train()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=True, position=0, desc='Train')

    running_loss        = 0.0
    running_perplexity  = 0.0
    
    for i, (x, y, lx, ly) in enumerate(dataloader):

        optimizer.zero_grad()

        x, y, lx, ly = x.to(DEVICE), y.to(DEVICE), lx, ly

        with torch.cuda.amp.autocast():

            predictions, attention_plot = model(x, lx, y= y, tf_rate= teacher_forcing_rate)

            # Predictions are of Shape (batch_size, timesteps, vocab_size). 
            batch_size, timesteps, vocab_size = predictions.shape

            # Transcripts are of shape (batch_size, timesteps) Which means that you have batch_size amount of batches with timestep number of tokens.
            # So in total, you have batch_size*timesteps amount of characters.
            # Similarly, in predictions, you have batch_size*timesteps amount of probability distributions.
            # How do you need to modify transcipts and predictions so that you can calculate the CrossEntropyLoss? Hint: Use Reshape/View and read the docs
            loss        = criterion(predictions.reshape(-1, vocab_size), y.reshape(-1)).to(DEVICE)# TODO: Cross Entropy Loss
            mask        = torch.arange(y.size(1)).unsqueeze(0) <= (ly).unsqueeze(1)# TODO: Create a boolean mask using the lengths of your transcript that remove the influence of padding indices (in transcripts) in the loss 
            mask        = mask.to(DEVICE)
            masked_loss = torch.sum(loss * mask.view(-1)) / torch.sum(mask)# Product between the mask and the loss, divided by the mask's sum. Hint: You may want to reshape the mask too 
            perplexity  = torch.exp(masked_loss) # Perplexity is defined the exponential of the loss

            running_loss        += masked_loss.item()
            running_perplexity  += perplexity.item()
        
        # Backward on the masked loss
        scaler.scale(masked_loss).backward()

        # Optional: Use torch.nn.utils.clip_grad_norm to clip gradients to prevent them from exploding, if necessary
        # If using with mixed precision, unscale the Optimizer First before doing gradient clipping
        
        scaler.step(optimizer)
        scaler.update()
        

        batch_bar.set_postfix(
            loss="{:.04f}".format(running_loss/(i+1)),
            perplexity="{:.04f}".format(running_perplexity/(i+1)),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
            tf_rate='{:.02f}'.format(teacher_forcing_rate))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    running_loss /= len(dataloader)
    running_perplexity /= len(dataloader)
    batch_bar.close()

    return running_loss, running_perplexity, attention_plot

In [ ]:
def validate(model, dataloader):

    model.eval()

    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=True, desc="Val")

    running_lev_dist = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):

        x, y, lx, ly = x.to(DEVICE), y.to(DEVICE), lx, ly

        with torch.inference_mode():
            predictions, attentions = model(x, lx, y = None)

        # Greedy Decoding
        greedy_predictions   =  torch.argmax(predictions,dim=2)# TODO: How do you get the most likely character from each distribution in the batch?

        # Calculate Levenshtein Distance
        running_lev_dist    += calc_edit_distance(greedy_predictions, y, ly, VOCAB, print_example = False) # You can use print_example = True for one specific index i in your batches if you want

        batch_bar.set_postfix(
            dist="{:.04f}".format(running_lev_dist/(i+1)))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    batch_bar.close()
    running_lev_dist /= len(dataloader)

    return running_lev_dist#, running_loss, running_perplexity, 

# Experiments

In [ ]:
best_lev_dist = float("inf")
tf_rate = 1

for epoch in range(0, 30):
    
    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    # Call train and validate 
    running_loss, running_perplexity, attention_plot = train(model, train_loader, criterion, optimizer, tf_rate)
    running_lev_dist = validate(model, val_loader)

    # Print your metrics

    # Plot Attention 
    plot_attention(attention_plot)

    # Log metrics to Wandb

    # Optional: Scheduler Step / Teacher Force Schedule Step
    scheduler.step(running_loss)

    if epoch>10:
      tf_rate -= 0.01

    if running_lev_dist <= best_lev_dist:
        best_lev_dist = running_lev_dist
        print('saving...')
        # Save your model checkpoint here
        torch.save(model.state_dict(), "model_best.pt")

# Testing

In [ ]:
# Optional: Load your best model Checkpoint here
model.load_state_dict(torch.load('/content/model_best.pt'))

<All keys matched successfully>

In [ ]:
# TODO: Create a testing function similar to validation 
def indices_to_string(indices, VOCAB):
    tokens = ""
    for i in indices:
        if int(i) == SOS_TOKEN: # If SOS is encountered, dont add it to the final list
            continue
        elif int(i) == EOS_TOKEN: # If EOS is encountered, stop the decoding process
            break
        else:
            tokens += VOCAB[i]
    return tokens

def test(model, dataloader):
    with torch.no_grad():
        model.eval()
        preds = []
        for i, (x, lx) in enumerate(dataloader):

            x, lx = x.to(DEVICE), lx

            with torch.inference_mode():
                predictions, attentions = model(x, lx, y = None)

            # Greedy Decoding
            greedy_predictions   =  torch.argmax(predictions,dim=2)# TODO: How do you get the most likely character from each distribution in the batch?
            print(greedy_predictions.shape)
            for i in range(len(x)):
                preds.append(indices_to_string(greedy_predictions[i], VOCAB))

            torch.cuda.empty_cache()
    return preds

In [ ]:
preds = test(model, test_loader)

NameError: ignored

In [ ]:
# TODO: Create a file with all predictions 
with open('submission.csv', 'w') as f:
  f.write('id,label\n')
  for i in range(len(preds)):
    f.write(str(i) + ',' + preds[i]+"\n")

NameError: ignored

In [ ]:
print(len(preds))

2620


In [ ]:
# TODO: Submit to Kaggle
!kaggle competitions submit -c 11-785-f22-hw4p2 -f submission.csv -m "I made it!"

100% 64.2k/64.2k [00:00<00:00, 99.4kB/s]
Successfully submitted to Attention-Based Speech Recognition